## Loading a United-atom cy5 representation into mbuild

In [1]:
import numpy as np
import mbuild as mb

/Users/stephaniemccallum/miniforge3/envs/flowermd-dev/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
dye = mb.load('../data/CY5-bonds.pdb')
dye.visualize().show()

/Users/stephaniemccallum/miniforge3/envs/flowermd-dev/lib/python3.12/site-packages/mbuild/compound.py:546: UserWarning: Some particle of <Compound 58 particles, 61 bonds, non-periodic, id: 6256924272> does not have a charge.They will not be accounted for during this calculation.
  warn(


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Loading cy5 into flowerMD Molecule

In [3]:
from flowermd.base import Molecule, System

In [5]:
dye = Molecule(num_mols=2,file='../data/CY5-bonds.pdb')

In [8]:
print(dye,dye.molecules)

<flowermd.base.molecule.Molecule object at 0x297bf5640> [<Compound 58 particles, 61 bonds, non-periodic, id: 11098527920>, <Compound 58 particles, 61 bonds, non-periodic, id: 11132975488>]


## make a molecule class that add tethers to the linker sites and removes hydrogens for UA model

TODO: write an initialization class based on system where we can control the placement of the dyes in a box.

make each dye a rigid body from snapshot

apply forcefield and exclude tether particles

maybe: add wall and wall potential to restrict rotation of dyes